## Learning ipywidgets: a very simple d3 "CircleWidget" with a js CircleView 

This simple notebook demonstrates:
  - the creation of a d3 canvas for displaying circles
  - python function calls to draw circles (with red borders)
  - mouse click to create circles directly on the d3 svg canvas (with blue borders)
  - syncing of the circleCount variable from browser back to kernel
  - two html/jQuery visual elements: a circleCount readout, a "clear all circles" button
  - state (the circles currently on the canvas) is recorded in javascript, not in python


[github conversation](https://github.com/ipython/ipywidgets/issues/838#issuecomment-255506936)|

## Learn how to use custom widgets for the Jupyter Notebook

In [107]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    },
});

<IPython.core.display.Javascript object>

In [108]:
import ipywidgets
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate

class CircleWidget(ipywidgets.DOMWidget):
    _view_name = Unicode('CircleView').tag(sync=True)
    _view_module = Unicode('circle').tag(sync=True)
    radius = Int(100).tag(sync=True)
    circles = Tuple().tag(sync=True)
    width = Int().tag(sync=True)
    height = Int().tag(sync=True)
    radius = Int().tag(sync=True)
    color = Unicode().tag(sync=True)
    def __init__(self, **kwargs):
        super(ipywidgets.DOMWidget, self).__init__(**kwargs)
        self.width = kwargs.get('width', 500)
        self.height = kwargs.get('height', 500)
        self.radius = 1
        self.color = "#ff0000"
    def drawCircle(self, x, y, fillColor="red", borderColor="black"):
        newCircle = {"x": x,  "y": y, "radius": self.radius * 10, "fillColor": fillColor, "borderColor": borderColor}
        self.circles = self.circles + (newCircle,)

In [109]:
%%javascript
"use strict";

require.undef('circle');

define('circle', ["@jupyter-widgets/base", "d3"], function(widgets, d3) {

    var CircleView = widgets.DOMWidgetView.extend({

        initialize: function() {
            console.log("---- initialize, this:");
            console.log(this);
            this.circles = [];
            this.radius = 1;
            this.color = "#ff0000";
            },

        createDiv: function(){
            var width = this.model.get('width');
            var height = this.model.get('height');
            var divstyle = $("<div id='d3DemoDiv' style='border:1px solid red; height: " +
                             height + "px; width: " + width + "px'>");
            return(divstyle);
            },

        createCanvas: function(){
            var width = this.model.get('width');
            var height = this.model.get('height');
            var radius = this.model.get('radius');
            console.log("--SIZE--", width, 'x', height, " radius", radius);
            var svg = d3.select("#d3DemoDiv")
                        .append("svg")
                        .attr("id", "svg").attr("width", width).attr("height", height)
                        .style("background-color", "yellow");

            this.svg = svg;
            var circleView = this;

            svg.on('click', function() {
                var coords = d3.mouse(this);
                //debugger;
                var radius = circleView.radius;
                var color  = circleView.color;
                console.log('--MOUSE--', coords, " radius:", radius);
                var newCircle = {x: coords[0], y: coords[1], radius: 10 * radius,
                                 borderColor: "black", fillColor: color};
                circleView.circles.push(newCircle);
                circleView.drawCircle(newCircle);
                //debugger;
                circleView.model.set("circles", JSON.stringify(circleView.circles));
                circleView.touch();
                });
           },

        drawCircle: function(obj){
           
           this.svg.append("circle")
              .style("stroke", "black")
              .style("fill", "white")
              .attr("r", obj.radius)
              .attr("cx", obj.x)
              .attr("cy", obj.y)
              .on("mouseover", function(){d3.select(this).style("fill", "aliceblue");})
              .on("mouseout",  function(){d3.select(this).style("fill", obj.fillColor);})
            },

        render: function() {
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:circles', this._circles_changed, this);
            this.listenTo(this.model, 'change:radius', this._radius_changed, this);
            this.listenTo(this.model, 'change:color', this._color_changed, this);
            var circleView = this;
            function myFunc(){
               circleView.createCanvas()
               };
            setTimeout(myFunc, 500);
            },

        _circles_changed: function() {
           var circles = this.model.get("circles");
           var newCircle = circles[circles.length-1];
           console.log('--DRAW--', this.circles);
           this.circles.push(newCircle);
           console.log('--LENGTH--', circles.length, " == ", circles.length);
           this.drawCircle(newCircle);
           },

        _radius_changed: function() {
           console.log('--RADIUS--', this.radius, this.model.get('radius'));
           this.radius = this.model.get('radius');
           },
        
        _color_changed: function() {
        this.color = this.model.get('color');
    }
    });
    return {
        CircleView : CircleView
    };
});

<IPython.core.display.Javascript object>

In [110]:
cw = CircleWidget(width=500, height=500)
scale = ipywidgets.IntSlider(1, 0, 10)
pcolor = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='blue',
    disabled=False
)
box = widgets.VBox([scale, cw, pcolor])
mylink1 = ipywidgets.jslink((cw, 'radius'), (scale, 'value'))
mylink2 = ipywidgets.jslink((cw, 'color'), (pcolor, 'value'))
box